# Importing necassary libraries

In [1]:
import numpy as np
import scipy.io
from python_speech_features import mfcc
import matplotlib.pyplot as plt
import IPython.display as ipd
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import soundfile as sf
import random
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import joblib
import math
import statistics




# Defining the function that reads the audios from a given path using soundfile and returns 4 lists : audios, freqs, filepaths and problematic files for potential debugging

In [2]:

def read_audios(directory):
    audios = []
    freqs = []
    filepaths = []
    problematic_files = []
    
    # Walking through the directory that contains the dataset and reading each file that has the .wav extension
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                filepath = os.path.join(root, file)
                filepaths.append(filepath)
                try:
                    # Read the audio file using soundfile
                    data, freq = sf.read(filepath)
                    audios.append(data)
                    freqs.append(freq)
                except Exception as e:
                    print(f"Encountered an error with file: {filepath}. Error: {e}")
                    problematic_files.append(filepath)
    
    return audios, freqs, filepaths, problematic_files

# Example usage
audios, freqs, filepaths, problematic_files = read_audios(r'turkish_wav')


# Defining the function that extracts the mfcc features then removes the frames of silence finally it saves the mffc features into a .txt file according to gender 

In [3]:
def extractMfccs_RemoveSilence_saveMfccs(audios,freqs,filepaths, directory):
    mfccs = []
 
    for audio, freq, filepath in zip(audios, freqs,filepaths):
        # extract the MFCC features
        mfcc_features = mfcc(audio, freq, winlen=0.025, winstep=0.01, numcep=13, nfilt=26, nfft= 2048, lowfreq=0,
                         highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=False)
        
        # calculate the energy
        energy = np.sum(mfcc_features**2, axis=1)
        # calculate the threshold for silence
        threshold = np.mean(energy) * 0.4
        #removing silence frames from mfccs
        voiced_indices = np.where(energy > threshold)[0]
        mfccs_voiced = mfcc_features[voiced_indices,:]
        mfccs.append(mfccs_voiced)
        
        # print the shape of the MFCCs before and after removing silence
        print(f"MFCCs before removing silence: {mfcc_features.shape}")
        print(f"MFCCs after removing silence: {mfccs_voiced.shape}")
        
       #saving mffcs 
       # extract the gender information from the file name
        gender = None
        if 'hommes' in filepath:
            gender = 'Hommes'
        elif 'femmes' in filepath:
            gender = 'Femmes'

        # save the MFCCs to the appropriate directory based on gender
        if gender is not None:
            gender_dir = os.path.join(directory, gender)
            if not os.path.exists(gender_dir):
                os.makedirs(gender_dir)
            mfcc_file = os.path.join(gender_dir,  os.path.splitext(os.path.basename(filepath))[0] + ".mfcc")
            np.savetxt(mfcc_file, mfccs_voiced, delimiter=',')
            
      
    
    return  mfccs

# Defining the function that splits the extraced mfccs into training and testing sets : 2/3 from male directory for training and 2/3 from female 

In [4]:
def train_test_split(mfcc_dir):
    # create separate lists for male and female file paths
    male_files = []
    female_files = []
    for root, dirs, files in os.walk(mfcc_dir):
        for file in files:
            if file.endswith('.mfcc'):
                if 'Hommes' in root:
                    male_files.append(os.path.join(root, file))
                elif 'Femmes' in root:
                    female_files.append(os.path.join(root, file))

    # shuffle the male and female lists independently
    random.shuffle(male_files)
    random.shuffle(female_files)

    # split the male and female lists into train and test based on the desired ratio
    male_train = male_files[:int(2/3*len(male_files))]
    male_test = male_files[int(2/3*len(male_files)):]

    female_train = female_files[:int(2/3*len(female_files))]
    female_test = female_files[int(2/3*len(female_files)):]

    
    
    # merge the train and test lists for both male and female
    train_files = male_train + female_train
    test_files = male_test + female_test

    # load the MFCC features from the saved files for the train and test sets
    train_mfccs = []
    test_mfccs = []

    for file in train_files:
        train_mfccs.append(np.loadtxt(file, delimiter=','))

    for file in test_files:
        test_mfccs.append(np.loadtxt(file, delimiter=','))

    # print the shapes of the train and test MFCC feature arrays
    print(f"Train male MFCCs shape: {np.array(male_train).shape}")
    print(f"Test male MFCCs shape: {np.array( male_test).shape}")
    print(f"Train female MFCCs shape: {np.array(female_train).shape}")
    print(f"Test female MFCCs shape: {np.array( female_test).shape}")
    print(f"Train MFCCs shape: {np.array(train_mfccs).shape}")
    print(f"Test MFCCs shape: {np.array(test_mfccs).shape}")
    
    return train_mfccs, test_mfccs


# Defining the functions that train the different GMM models and than save them as a pkl file

In [5]:
def gmm16(train_mfccs):
    # Initialize the GMM model with 16 classes
    gmm = GaussianMixture(n_components=16, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model16_turkish.pkl')

    return gmm


In [6]:
def gmm32(train_mfccs):
    # Initialize the GMM model with 32 classes
    gmm = GaussianMixture(n_components=32, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model32_turkish.pkl')

    return gmm

In [7]:
def gmm64(train_mfccs):
    # Initialize the GMM model with 64 classes
    gmm = GaussianMixture(n_components=64, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model64_turkish.pkl')

    return gmm

In [8]:
def gmm128(train_mfccs):
    # Initialize the GMM model with 128 classes
    gmm = GaussianMixture(n_components=128, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model128_turkish.pkl')

    return gmm

In [9]:
def gmm256(train_mfccs):
    # Initialize the GMM model with 254 classes
    gmm = GaussianMixture(n_components=256, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model256_turkish.pkl')

    return gmm

# Getting the audios frequencies and filepaths from their directory using the function defined above

In [10]:
audios, freqs, filepaths, problematic_files = read_audios(r'turkish_wav')


# Extracting the mfcc features - Removing silence and saving the mfccs

In [11]:
mfccs = extractMfccs_RemoveSilence_saveMfccs(audios, freqs, filepaths,r'mfcc_turkish')

MFCCs before removing silence: (1821, 13)
MFCCs after removing silence: (1759, 13)
MFCCs before removing silence: (1842, 13)
MFCCs after removing silence: (1773, 13)
MFCCs before removing silence: (1389, 13)
MFCCs after removing silence: (1359, 13)
MFCCs before removing silence: (1590, 13)
MFCCs after removing silence: (1514, 13)
MFCCs before removing silence: (1425, 13)
MFCCs after removing silence: (1415, 13)
MFCCs before removing silence: (978, 13)
MFCCs after removing silence: (962, 13)
MFCCs before removing silence: (1209, 13)
MFCCs after removing silence: (1137, 13)
MFCCs before removing silence: (1043, 13)
MFCCs after removing silence: (963, 13)
MFCCs before removing silence: (1050, 13)
MFCCs after removing silence: (1028, 13)
MFCCs before removing silence: (1655, 13)
MFCCs after removing silence: (1599, 13)
MFCCs before removing silence: (1331, 13)
MFCCs after removing silence: (1261, 13)
MFCCs before removing silence: (2001, 13)
MFCCs after removing silence: (1919, 13)
MFCCs b

MFCCs before removing silence: (1569, 13)
MFCCs after removing silence: (1514, 13)
MFCCs before removing silence: (1439, 13)
MFCCs after removing silence: (1401, 13)
MFCCs before removing silence: (1835, 13)
MFCCs after removing silence: (1805, 13)
MFCCs before removing silence: (1446, 13)
MFCCs after removing silence: (1348, 13)
MFCCs before removing silence: (827, 13)
MFCCs after removing silence: (803, 13)
MFCCs before removing silence: (1547, 13)
MFCCs after removing silence: (1519, 13)
MFCCs before removing silence: (1677, 13)
MFCCs after removing silence: (1636, 13)
MFCCs before removing silence: (1662, 13)
MFCCs after removing silence: (1611, 13)
MFCCs before removing silence: (993, 13)
MFCCs after removing silence: (948, 13)
MFCCs before removing silence: (1461, 13)
MFCCs after removing silence: (1443, 13)
MFCCs before removing silence: (1806, 13)
MFCCs after removing silence: (1768, 13)
MFCCs before removing silence: (1122, 13)
MFCCs after removing silence: (1063, 13)
MFCCs be

MFCCs before removing silence: (1086, 13)
MFCCs after removing silence: (1083, 13)
MFCCs before removing silence: (1626, 13)
MFCCs after removing silence: (1624, 13)
MFCCs before removing silence: (741, 13)
MFCCs after removing silence: (713, 13)
MFCCs before removing silence: (1137, 13)
MFCCs after removing silence: (1134, 13)
MFCCs before removing silence: (1806, 13)
MFCCs after removing silence: (1799, 13)
MFCCs before removing silence: (1389, 13)
MFCCs after removing silence: (1386, 13)
MFCCs before removing silence: (1446, 13)
MFCCs after removing silence: (1446, 13)
MFCCs before removing silence: (906, 13)
MFCCs after removing silence: (900, 13)
MFCCs before removing silence: (1050, 13)
MFCCs after removing silence: (1046, 13)
MFCCs before removing silence: (1461, 13)
MFCCs after removing silence: (1458, 13)
MFCCs before removing silence: (1533, 13)
MFCCs after removing silence: (1526, 13)
MFCCs before removing silence: (1007, 13)
MFCCs after removing silence: (993, 13)
MFCCs bef

MFCCs before removing silence: (1151, 13)
MFCCs after removing silence: (1148, 13)
MFCCs before removing silence: (798, 13)
MFCCs after removing silence: (794, 13)
MFCCs before removing silence: (1425, 13)
MFCCs after removing silence: (1420, 13)
MFCCs before removing silence: (1662, 13)
MFCCs after removing silence: (1659, 13)
MFCCs before removing silence: (1331, 13)
MFCCs after removing silence: (1331, 13)
MFCCs before removing silence: (1583, 13)
MFCCs after removing silence: (1580, 13)
MFCCs before removing silence: (1317, 13)
MFCCs after removing silence: (1314, 13)
MFCCs before removing silence: (1677, 13)
MFCCs after removing silence: (1674, 13)
MFCCs before removing silence: (1497, 13)
MFCCs after removing silence: (1494, 13)
MFCCs before removing silence: (741, 13)
MFCCs after removing silence: (739, 13)
MFCCs before removing silence: (654, 13)
MFCCs after removing silence: (640, 13)
MFCCs before removing silence: (690, 13)
MFCCs after removing silence: (681, 13)
MFCCs before

MFCCs before removing silence: (546, 13)
MFCCs after removing silence: (545, 13)
MFCCs before removing silence: (561, 13)
MFCCs after removing silence: (552, 13)
MFCCs before removing silence: (453, 13)
MFCCs after removing silence: (451, 13)
MFCCs before removing silence: (1158, 13)
MFCCs after removing silence: (1157, 13)
MFCCs before removing silence: (971, 13)
MFCCs after removing silence: (958, 13)
MFCCs before removing silence: (813, 13)
MFCCs after removing silence: (722, 13)
MFCCs before removing silence: (1209, 13)
MFCCs after removing silence: (1138, 13)
MFCCs before removing silence: (798, 13)
MFCCs after removing silence: (708, 13)
MFCCs before removing silence: (1101, 13)
MFCCs after removing silence: (1019, 13)
MFCCs before removing silence: (669, 13)
MFCCs after removing silence: (588, 13)
MFCCs before removing silence: (1115, 13)
MFCCs after removing silence: (1094, 13)
MFCCs before removing silence: (1158, 13)
MFCCs after removing silence: (1140, 13)
MFCCs before remov

MFCCs before removing silence: (546, 13)
MFCCs after removing silence: (545, 13)
MFCCs before removing silence: (582, 13)
MFCCs after removing silence: (580, 13)
MFCCs before removing silence: (582, 13)
MFCCs after removing silence: (565, 13)
MFCCs before removing silence: (633, 13)
MFCCs after removing silence: (628, 13)
MFCCs before removing silence: (1209, 13)
MFCCs after removing silence: (1189, 13)
MFCCs before removing silence: (683, 13)
MFCCs after removing silence: (679, 13)
MFCCs before removing silence: (834, 13)
MFCCs after removing silence: (795, 13)
MFCCs before removing silence: (582, 13)
MFCCs after removing silence: (582, 13)
MFCCs before removing silence: (726, 13)
MFCCs after removing silence: (711, 13)
MFCCs before removing silence: (957, 13)
MFCCs after removing silence: (919, 13)
MFCCs before removing silence: (633, 13)
MFCCs after removing silence: (632, 13)
MFCCs before removing silence: (568, 13)
MFCCs after removing silence: (565, 13)
MFCCs before removing sile

MFCCs before removing silence: (381, 13)
MFCCs after removing silence: (257, 13)
MFCCs before removing silence: (474, 13)
MFCCs after removing silence: (375, 13)
MFCCs before removing silence: (373, 13)
MFCCs after removing silence: (238, 13)
MFCCs before removing silence: (366, 13)
MFCCs after removing silence: (253, 13)
MFCCs before removing silence: (373, 13)
MFCCs after removing silence: (271, 13)
MFCCs before removing silence: (373, 13)
MFCCs after removing silence: (287, 13)
MFCCs before removing silence: (366, 13)
MFCCs after removing silence: (278, 13)
MFCCs before removing silence: (345, 13)
MFCCs after removing silence: (206, 13)
MFCCs before removing silence: (402, 13)
MFCCs after removing silence: (302, 13)
MFCCs before removing silence: (417, 13)
MFCCs after removing silence: (339, 13)
MFCCs before removing silence: (611, 13)
MFCCs after removing silence: (601, 13)
MFCCs before removing silence: (669, 13)
MFCCs after removing silence: (610, 13)
MFCCs before removing silenc

MFCCs before removing silence: (561, 13)
MFCCs after removing silence: (537, 13)
MFCCs before removing silence: (719, 13)
MFCCs after removing silence: (680, 13)
MFCCs before removing silence: (633, 13)
MFCCs after removing silence: (629, 13)
MFCCs before removing silence: (1425, 13)
MFCCs after removing silence: (1403, 13)
MFCCs before removing silence: (611, 13)
MFCCs after removing silence: (571, 13)
MFCCs before removing silence: (525, 13)
MFCCs after removing silence: (525, 13)
MFCCs before removing silence: (575, 13)
MFCCs after removing silence: (549, 13)
MFCCs before removing silence: (1122, 13)
MFCCs after removing silence: (1095, 13)
MFCCs before removing silence: (1187, 13)
MFCCs after removing silence: (1097, 13)
MFCCs before removing silence: (1468, 13)
MFCCs after removing silence: (1455, 13)
MFCCs before removing silence: (1389, 13)
MFCCs after removing silence: (1375, 13)
MFCCs before removing silence: (1396, 13)
MFCCs after removing silence: (1386, 13)
MFCCs before rem

MFCCs before removing silence: (1187, 13)
MFCCs after removing silence: (1157, 13)
MFCCs before removing silence: (777, 13)
MFCCs after removing silence: (772, 13)
MFCCs before removing silence: (1403, 13)
MFCCs after removing silence: (1389, 13)
MFCCs before removing silence: (1259, 13)
MFCCs after removing silence: (1209, 13)
MFCCs before removing silence: (1259, 13)
MFCCs after removing silence: (1249, 13)
MFCCs before removing silence: (1101, 13)
MFCCs after removing silence: (1085, 13)
MFCCs before removing silence: (582, 13)
MFCCs after removing silence: (575, 13)
MFCCs before removing silence: (1410, 13)
MFCCs after removing silence: (1394, 13)
MFCCs before removing silence: (1223, 13)
MFCCs after removing silence: (1218, 13)
MFCCs before removing silence: (1230, 13)
MFCCs after removing silence: (1223, 13)
MFCCs before removing silence: (863, 13)
MFCCs after removing silence: (842, 13)
MFCCs before removing silence: (777, 13)
MFCCs after removing silence: (734, 13)
MFCCs before

## -----> We can see here that the size of the mfcc features has decreased after removing the frames ot silence

# Splitting into test and train sets according to gender

In [12]:
train_mfccs, test_mfccs = train_test_split(r'mfcc_turkish')

Train male MFCCs shape: (332,)
Test male MFCCs shape: (167,)
Train female MFCCs shape: (272,)
Test female MFCCs shape: (136,)
Train MFCCs shape: (604,)
Test MFCCs shape: (303,)


C:\Users\HP\AppData\Local\Temp\ipykernel_14664\3397580047.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f"Train MFCCs shape: {np.array(train_mfccs).shape}")
C:\Users\HP\AppData\Local\Temp\ipykernel_14664\3397580047.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f"Test MFCCs shape: {np.array(test_mfccs).shape}")


# Stacking vertically the train and test MFCC features so that we can fit the gmm models

In [13]:
#stack vertically the train MFCC features 
mfcc_train = []
for train_mfcc in train_mfccs:
    mfcc_train.append(train_mfcc)
mfcc_train = np.concatenate(mfcc_train, axis=0)

#stack vertically the test MFCC features 
mfcc_test = []
for test_mfcc in test_mfccs:
    mfcc_test.append(test_mfcc)
mfcc_test = np.concatenate(mfcc_test, axis=0)


# Saving the test set into a txt file 

In [14]:
#Save the test mfccs in a file
test_mfccs = np.vstack(test_mfccs)
test_mfccs = np.array(test_mfccs, dtype=float)
np.savetxt(r'turkishTest', test_mfccs, delimiter=',')



In [15]:
mfcc_train.shape

(576259, 13)

# Training the different Gmm Models 

In [16]:
gmm16 = gmm16(mfcc_train)

In [17]:
gmm32 = gmm32(mfcc_train)

In [18]:
gmm64 = gmm64(mfcc_train)

In [19]:
gmm128= gmm128(mfcc_train)

In [20]:
gmm256= gmm256(mfcc_train)

# Evaluate the performance of each GMM model on the test set using the score_samples() function that returns an array containing the log-likelihood of each frame of the mfcc features

In [21]:
scores = []
for model in [gmm16, gmm32, gmm64, gmm128, gmm256]:
    score = model.score_samples(mfcc_test)
    scores.append(score)

# Print the scores
print('GMM16 score:', scores[0])
print('GMM32 score:', scores[1])
print('GMM64 score:', scores[2])
print('GMM128 score:', scores[3])
print('GMM256 score:', scores[4])



GMM16 score: [ 73.0116136   73.0116136   73.0116136  ... -49.36020714 -51.3305941
 -50.78653811]
GMM32 score: [ 73.0116136   73.0116136   73.0116136  ... -49.8530643  -49.24600027
 -48.88756001]
GMM64 score: [ 73.0116136   73.0116136   73.0116136  ... -44.80738092 -47.62213518
 -45.68116523]
GMM128 score: [ 73.0116136   73.0116136   73.0116136  ... -45.56326282 -46.52130005
 -43.34020428]
GMM256 score: [ 73.0116136   73.0116136   73.0116136  ... -46.02174851 -47.34923214
 -41.25944269]


# Comparing the size of our mfcc_test set with the size of the scores array 

In [22]:
mfcc_test.shape

(279532, 13)

# Indeed the size of the scores array is the same 

In [23]:
 scores[0].shape

(279532,)

# In order to compare between the different GMM Models we need to calculate the score for the whole test set and we can do that by calculating the mean of the individual scores

In [24]:
#calculationg the score of the hole test set
print('GMM16 score:', scores[0].mean())
print('GMM32 score:', scores[1].mean())
print('GMM64 score:', scores[2].mean())
print('GMM128 score:', scores[3].mean())
print('GMM256 score:', scores[4].mean())


GMM16 score: -48.58593198178585
GMM32 score: -48.00377450279551
GMM64 score: -47.477795502404675
GMM128 score: -47.02006404197736
GMM256 score: -46.621705838911474


### From the results above we can see that the best score (the closest one to 0) is given by the model using 16 gaussians 